# Financial Analyst Chatbot


## Overview

RAG-powered chatbot that enables natural language querying of BNP Paribas Q2 2025 earnings data, providing instant access to financial metrics and insights from earnings calls and press releases.

## Business Use Case

Financial analysts spend hours manually extracting insights from lengthy earnings reports. This solution reduces analysis time from hours to minutes while maintaining accuracy and providing full source traceability for compliance.

## Notebook Steps

1. **Environment Setup** - Install dependencies and configure Mistral API
2. **Document Processing** - Load and chunk BNP earnings documents
3. **Vector Database Creation** - Generate embeddings and build FAISS index
4. **RAG Pipeline Implementation** - Build retrieval and generation components
5. **Interactive Testing** - Query the system and validate responses with source citations


## Environment Setup


Let's install all required libraries for this project


In [8]:
from dotenv import load_dotenv
import os
import requests
from mistralai import Mistral
import numpy as np

load_dotenv()

True

Configure the Mistral AI API KEY


In [ ]:
# if you have a .env file with MISTRAL_API key
MISTRAL_API_KEY = os.getenv("MISTRAL_API")

# MISTRAL_API_KEY = ""


client = Mistral(api_key=MISTRAL_API_KEY)

## Document Processing


### Data Ingestion


We process three key BNP Paribas Q2 2025 documents: the earnings call transcript and press release materials (split into text content and financial statements for optimized processing). Using LangChain's PyPDFLoader, we extract text while preserving crucial metadata like source attribution and page numbers, essential for providing accurate citations in our RAG responses.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

docs = []
data_dir = Path("data")
file_map = {
    "2Q25_PressRelease_Graphs.pdf": "BNP 2Q25 Press Release Graphs",
    "2Q25_call_transcript.pdf": "BNP 2Q25 Earnings Call Transcript",
    "2Q25_PressRelease.pdf": "BNP 2Q25 Press Release",
}

for file_name, source_name in file_map.items():
    file_path = data_dir / file_name
    loader = PyPDFLoader(str(file_path))
    loaded_docs = loader.load()
    for doc in loaded_docs:
        doc.metadata["source"] = source_name
    docs.extend(loaded_docs)

### Chunk Documents


We implement a recursive character-based chunking strategy optimized for financial document structure and retrieval performance.

**Chunking Strategy Rationale:**

- **Chunk size (1200 chars)**: Optimal balance for financial documents -> captures complete financial statements and contextual paragraphs without exceeding token limits
- **Overlap (200 chars)**: Ensures critical information spanning chunk boundaries (like multi-line financial metrics) isn't lost during retrieval
- **Hierarchical separators**: Prioritizes natural document structure - paragraphs first, then sentences, then words - maintaining semantic coherence in financial content

**Note:** This approach works well for our demo, but production systems could benefit from advanced techniques like semantic chunking or domain-specific boundary detection for enhanced retrieval accuracy.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", "", ".", "?", "!"],
)
chunks = splitter.split_documents(docs)  # This preserves metadata from docs

# Verify metadata is preserved
print(f"Total chunks created: {len(chunks)}")
print(f"Sample chunk metadata: {chunks[0].metadata}")
print("Sources found in chunks:")
for source in set(chunk.metadata.get("source", "Unknown") for chunk in chunks):
    count = sum(1 for chunk in chunks if chunk.metadata.get("source") == source)
    print(f"  - {source}: {count} chunks")

Total chunks created: 151
Sample chunk metadata: {'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250809105600', 'source': 'BNP 2Q25 Press Release Graphs', 'total_pages': 4, 'page': 0, 'page_label': '1'}
Sources found in chunks:
  - BNP 2Q25 Press Release: 75 chunks
  - BNP 2Q25 Earnings Call Transcript: 67 chunks
  - BNP 2Q25 Press Release Graphs: 9 chunks


## Vector Index Creation


### Embedding our Data


We generate vector embeddings using Mistral's embedding model to enable semantic similarity search across our financial document chunks.

**Rate Limiting Strategy:** The custom retry function addresses Mistral API rate limits by implementing batch processing and exponential backoff with jitter, ensuring reliable embedding generation without manual intervention during processing.


In [ ]:
import time
import random
from typing import List


def get_embeddings_with_retry(
    inputs: List[str], max_retries: int = 5, batch_size: int = 10
):
    """
    Get embeddings with exponential backoff retry and batch processing
    """
    all_embeddings = []

    # Process in batches to reduce API calls
    for i in range(0, len(inputs), batch_size):
        batch = inputs[i : i + batch_size]

        for attempt in range(max_retries):
            try:
                # Make API call for the batch
                embeddings_batch_response = client.embeddings.create(
                    model="mistral-embed", inputs=batch
                )

                # Extract embeddings from response
                batch_embeddings = [
                    item.embedding for item in embeddings_batch_response.data
                ]
                all_embeddings.extend(batch_embeddings)
                break

            except Exception as e:
                if "rate_limited" in str(e) or "429" in str(e):
                    if attempt < max_retries - 1:
                        # Exponential backoff with jitter
                        wait_time = (2**attempt) + random.uniform(0, 1)
                        print(
                            f"Rate limited. Waiting {wait_time:.1f}s before retry {attempt + 1}/{max_retries}"
                        )
                        time.sleep(wait_time)
                    else:
                        print(f"Max retries exceeded for batch starting at index {i}")
                        raise e
                else:
                    print(f"Non-rate-limit error: {e}")
                    raise e

        # Small delay between batches
        if i + batch_size < len(inputs):
            time.sleep(0.5)

    print(f"Successfully processed {len(all_embeddings)} embeddings")

    return all_embeddings


# Convert chunks to text list
chunk_texts = [chunk.page_content for chunk in chunks]

print(f"Processing {len(chunk_texts)} chunks in batches...")
embeddings_list = get_embeddings_with_retry(chunk_texts, batch_size=8)
text_embeddings = np.array(embeddings_list)

Processing 151 chunks in batches...
Successfully processed 151 embeddings


## Vector Index Creation & Test


We create a FAISS index to enable fast semantic similarity queries across our embedded document chunks.


In [ ]:
import faiss


# Build vector search index for semantic similarity

embedding_dimension = text_embeddings.shape[1]

similarity_index = faiss.IndexFlatL2(embedding_dimension)

similarity_index.add(text_embeddings)

Export the faiss index for Demo Application


In [ ]:
# Export FAISS Index and Chunks for Streamlit App
import pickle
import os

# Create exports directory
os.makedirs("faiss_index", exist_ok=True)

# Export FAISS index
faiss.write_index(similarity_index, "faiss_index/faiss_index.bin")
print("✅ Exported FAISS index to faiss_index/faiss_index.bin")

# Export chunks with text content and metadata (needed for RAG)
with open("faiss_index/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)
print("✅ Exported chunks to faiss_index/chunks.pkl")

### Retrieval & Source Attribution


We implement semantic search to find the most relevant document chunks and format them with complete source attribution for accurate citations.

**Retrieval Process:**

- **Query embedding**: Convert user question to vector using same Mistral Embed model for consistency
- **Similarity search**: FAISS L2 distance search returns top-k most semantically similar chunks (k=5)
- **Metadata preservation**: Each retrieved chunk maintains source document, page number, and content information
- **Structured formatting**: Chunks formatted with clear source attribution (document name + page) for citation transparency

**Citation Strategy:** The formatted output provides complete traceability from answer back to source document and specific page, essential for financial compliance and fact-checking requirements in enterprise environments.


In [ ]:
# Query: find most relevant document chunks
user_question = "What was the Group's net income expected for 2025?"

# Use the same retry mechanism for query embeddings
question_embeddings = get_embeddings_with_retry([user_question], batch_size=1)
question_embedding = np.array([question_embeddings[0]])

# Search for the most similar chunks using L2 distance
distances, chunk_indices = similarity_index.search(question_embedding, k=5)

# Retrieve the actual text chunks with their metadata
most_relevant_chunks = [chunks[idx] for idx in chunk_indices[0]]


# Format chunks with source information for better citations
def format_chunk_with_source(chunk, chunk_num):
    source = chunk.metadata.get("source", "Unknown Source")
    page = chunk.metadata.get("page", "Unknown Page")

    return f"""
--- CHUNK {chunk_num + 1} ---
Source: {source} (Page {page + 1})
Content: {chunk.page_content}
---
"""


formatted_chunks = [
    format_chunk_with_source(chunk, i) for i, chunk in enumerate(most_relevant_chunks)
]

# Display the retrieved chunks with sources
print("Retrieved chunks with sources:")
for formatted_chunk in formatted_chunks:
    print(formatted_chunk)

Successfully processed 1 embeddings
Retrieved chunks with sources:

--- CHUNK 1 ---
Source: BNP 2Q25 Press Release (Page 3)
Content: In 2Q25, stage 3 provisions were contained at €830m (€1,027m in 2Q24), while provisions on 
performing loans came to €54m this quarter. As of 30.06.2025, the stock of provisions came to 
€18.2bn, including €4.1bn in stage 1 and 2 provisions. The stage 3 coverage ratio stood at 68.8% 
on a ratio of non-performing loans of 1.6%.  
The other net losses for risk on financial instruments amounted to €100m in 2Q25 (€91m in 2Q24). 
Operating income, pre-tax income and net income, Group share 
Group operating income came to €4,365m, up by 2.7% compared to 2Q24 (€4,251m). 
Net income, Group share  amounted to €3,258m in 2Q25, down by 4.0% compared to 2Q24 
(€3,395m). For reference, the corporate income tax rate in 2Q24 was low (20.8%) due to a 2Q24 
change in the method of taxing financing costs in the United States.  The average corporate 
income tax rate stood a

## RAG Pipeline


### Generation Basic Answer


We construct a detailed prompt that combines retrieved context with specific instructions for accurate, cited financial analysis responses.

**Prompt Engineering Strategy:**

- **Role definition**: Establishes AI as financial analysis specialist to prime domain-specific responses
- **Context injection**: Inserts retrieved chunks with preserved source metadata directly into prompt
- **Citation requirements**: Explicit instructions for proper source attribution in format [Source: Document Name (Page X)]
- **Boundary constraints**: Restricts answers to provided context only, preventing hallucination of financial data
- **Transparency mandate**: Requires clear statement when information is unavailable in context

**Model Selection:** Using Mistral Large Latest for superior reasoning capabilities on complex financial queries and reliable citation following - critical for maintaining accuracy in financial analysis applications.


In [ ]:
prompt = f"""You are an AI Assistant specialised in Financial Analysis. You are given a question and context information with sources. You need to answer the question based on the context and provide proper source citations.

Context information with sources is below:
---------------------
{''.join(formatted_chunks)}
---------------------

INSTRUCTIONS:
1. Answer the query based ONLY on the provided context information
2. For each fact or figure you mention, include a citation in the format [Source: Document Name (Page X)]
3. If information comes from multiple sources, cite all relevant sources
4. Be specific about which information comes from which source
5. If you cannot find information in the provided context, state that clearly

Query: {user_question}

Answer with proper source citations:
"""


def run_mistral(user_message, model="mistral-large-latest"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = client.chat.complete(model=model, messages=messages)
    return chat_response.choices[0].message.content


answer = run_mistral(prompt)

### Source Summary & Transparency


We provide a consolidated summary of all sources referenced in the response, enhancing transparency and enabling easy verification of information sources.

**Source Tracking Features:**

- **Automatic aggregation**: Collects all unique sources and page numbers from retrieved chunks
- **Deduplication**: Removes duplicate page references within each source document
- **Sorted presentation**: Orders page numbers for easy navigation and verification
- **Clear formatting**: Professional summary format with document names and specific page ranges
- **Audit trail**: Complete traceability for compliance and fact-checking requirements

This source summary enables financial analysts to quickly verify information, supports regulatory compliance, and builds trust in AI-generated insights by providing complete transparency into the underlying data sources.


In [ ]:
# Helper function to create source summary
def create_source_summary(chunks):
    """Create a summary of sources used in the response"""
    sources = {}
    for chunk in chunks:
        source_name = chunk.metadata.get("source", "Unknown Source")
        page = chunk.metadata.get("page", "Unknown Page")

        if source_name not in sources:
            sources[source_name] = set()
        sources[source_name].add(page + 1 if isinstance(page, int) else page)

    summary = "\n📚 SOURCES REFERENCED:\n"
    summary += "=" * 50 + "\n"

    for source, pages in sources.items():
        if isinstance(list(pages)[0], int):
            sorted_pages = sorted(list(pages))
            pages_str = ", ".join(map(str, sorted_pages))
        else:
            pages_str = ", ".join(map(str, pages))
        summary += f"• {source}: Pages {pages_str}\n"

    return summary


# Display source summary
print(create_source_summary(most_relevant_chunks))


📚 SOURCES REFERENCED:
• BNP 2Q25 Press Release: Pages 1, 3, 4
• BNP 2Q25 Earnings Call Transcript: Pages 3



### RAG - Complete Pipeline


We integrate all components into a function that delivers comprehensive financial analysis with full source attribution and metadata tracking.

**Pipeline Integration:**

- **End-to-end workflow**: Single function handles embedding, retrieval, generation, and citation in one seamless process
- **Configurable retrieval**: Adjustable top-k parameter allows fine-tuning relevance vs. context breadth
- **Structured output**: Returns organized response with answer, source summary, and operational metadata


In [ ]:
# Complete RAG Pipeline with Citations
def rag_with_citations(question, top_k=5):
    """
    Complete RAG pipeline with source citations

    Args:
        question (str): User's question
        top_k (int): Number of chunks to retrieve

    Returns:
        dict: Contains answer, sources, and metadata
    """

    # Get embeddings for the question
    question_embeddings = get_embeddings_with_retry([question], batch_size=1)
    question_embedding = np.array([question_embeddings[0]])

    # Search for most similar chunks
    distances, chunk_indices = similarity_index.search(question_embedding, k=top_k)
    relevant_chunks = [chunks[idx] for idx in chunk_indices[0]]

    # Format chunks with source information
    formatted_chunks = [
        format_chunk_with_source(chunk, i) for i, chunk in enumerate(relevant_chunks)
    ]

    # Create enhanced prompt
    enhanced_prompt = f"""You are an AI Assistant specialised in Financial Analysis. You are given a question and context information with sources. You need to answer the question based on the context and provide proper source citations.

Context information with sources is below:
---------------------
{''.join(formatted_chunks)}
---------------------

INSTRUCTIONS:
1. Answer the query based ONLY on the provided context information
2. For each fact or figure you mention, include a citation in the format [Source: Document Name (Page X)]
3. If information comes from multiple sources, cite all relevant sources
4. Be specific about which information comes from which source
5. If you cannot find information in the provided context, state that clearly

Query: {question}

Answer with proper source citations:
"""

    # Get answer with citations
    answer = run_mistral(enhanced_prompt)

    # Create source summary
    source_summary = create_source_summary(relevant_chunks)

    return {
        "question": question,
        "answer": answer,
        "source_summary": source_summary,
        "chunks_used": len(relevant_chunks),
        "sources": list(
            set(chunk.metadata.get("source", "Unknown") for chunk in relevant_chunks)
        ),
    }

### Complete Pipeline Example


In [ ]:
# Example usage of the complete pipeline
print("🚀 TESTING COMPLETE RAG PIPELINE WITH CITATIONS")
print("=" * 70)

test_question = "What was the Group's net income expected for 2025?"
result = rag_with_citations(test_question, top_k=3)

print(f"Question: {result['question']}")
print(f"Chunks used: {result['chunks_used']}")
print(f"Sources: {', '.join(result['sources'])}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

🚀 TESTING COMPLETE RAG PIPELINE WITH CITATIONS
Successfully processed 1 embeddings
Question: What was the Group's net income expected for 2025?
Chunks used: 3
Sources: BNP 2Q25 Press Release

Answer:
Based on the provided context, there is no explicit information about the Group's expected net income for the full year 2025. The context provides details about the net income for specific periods (2Q25 and 1H25) but does not include projections or expectations for the entire year of 2025.

Here are the relevant figures mentioned in the context:
- Net income, Group share for 2Q25: €3,258m [Source: BNP 2Q25 Press Release (Page 3)]
- Net income, Group share for 1H25: €6,209m [Source: BNP 2Q25 Press Release (Page 4)]

If you need the expected net income for the full year 2025, additional information beyond the provided context would be required.

📚 SOURCES REFERENCED:
• BNP 2Q25 Press Release: Pages 3, 4



## Usage Example


#### Simple Fact Retrieval


In [ ]:
question = "What was the Group's net income expected for 2025?"
# expected answer :
#  2025 is to exceed €12.2 billion
# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: What was the Group's net income expected for 2025?

Answer:
The Group's net income expected for 2025 is above €12.2 billion. This outlook is based on the solid second-quarter operating performances and the expected robust acceleration in the second half of the year [Source: BNP 2Q25 Press Release (Page 1)][Source: BNP 2Q25 Earnings Call Transcript (Page 3)].

The earnings call transcript further confirms this expectation, emphasizing that the net profit for 2025 is projected to exceed €12.2 billion, driven by strong revenue growth in the second half of the year [Source: BNP 2Q25 Earnings Call Transcript (Page 3)].

📚 SOURCES REFERENCED:
• BNP 2Q25 Press Release: Pages 1, 3, 4
• BNP 2Q25 Earnings Call Transcript: Pages 3



#### Specific Data Point Extraction


In [ ]:
question = "What was the interim dividend per share announced for 2025, and when will it be paid?"

# expected answer :
#  The interim dividend announced for 2025 is €2.59 per share. It will be paid out on 30 September 2025.
# generated answer :
result = rag_with_citations(question, top_k=3)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: What was the interim dividend per share announced for 2025, and when will it be paid?

Answer:
The interim dividend per share announced for 2025 is **€2.59**, and it will be paid on **30 September 2025**.

- The interim dividend amount of **€2.59 per share** is cited in:
  - [Source: BNP 2Q25 Press Release (Page 4)]
  - [Source: BNP 2Q25 Press Release (Page 1)]
  - [Source: BNP 2Q25 Earnings Call Transcript (Page 8)]

- The payment date of **30 September 2025** is cited in:
  - [Source: BNP 2Q25 Press Release (Page 4)]
  - [Source: BNP 2Q25 Press Release (Page 1)]
  - [Source: BNP 2Q25 Earnings Call Transcript (Page 8)]

📚 SOURCES REFERENCED:
• BNP 2Q25 Press Release: Pages 1, 4
• BNP 2Q25 Earnings Call Transcript: Pages 8



In [ ]:
question = "What was the CET1 ratio as of the end of the quarter, and how does management describe its stability?"
# expected answer :
# "Page 3. "Our CET1 is stable quarter on quarter at 12.5%". Page 6 expands on this: "stable quarter on quarter at 12.5%... well on track to be above our orientation of 12.3% CET1 pre-FRTB."# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: What was the CET1 ratio as of the end of the quarter, and how does management describe its stability?

Answer:
As of the end of the second quarter of 2025, the Common Equity Tier 1 (CET1) ratio was **12.5%** [Source: BNP 2Q25 Press Release (Page 5), Source: BNP 2Q25 Earnings Call Transcript (Page 6)].

Management describes the CET1 ratio as **stable quarter-on-quarter**, highlighting that it remains **far above the SREP requirements (10.48%)** and reflects **little volatility through the cycle** due to the Group's diversified business model. The stability is attributed to:
1. **Organic capital generation** (+20 basis points, offset by earnings distribution and model updates) [Source: BNP 2Q25 Press Release (Page 5)].
2. **RWA (Risk-Weighted Assets) optimization efforts**, which have contributed to cumulative gains of about **65 basis points** on the CET1 ratio [Source: BNP 2Q25 Press Release (Page 5)].
3. **Tight control of capital trajecto

#### Strategic Summary


In [ ]:
question = "Summarize the bank's 2026 growth trajectory as outlined in the reports."
# expected answer : Page  3, Earning Call Transcript under "2026 TRAJECTORY". It lists targets for RoTE, net income growth, and EPS growth
# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: Summarize the bank's 2026 growth trajectory as outlined in the reports.

Answer:
Based on the provided context, the bank's 2026 growth trajectory is outlined as follows:

1. **Return on Tangible Equity (RoTE)**:
   - The bank targets a RoTE of **12% in 2026**, up from 11.5% in 2025 [Source: BNP 2Q25 Earnings Call Transcript (Page 3)].
   - This is described as a "stepping stone towards further improvement" [Source: BNP 2Q25 Earnings Call Transcript (Page 3)].

2. **Revenue Growth**:
   - The bank expects a **compound annual growth rate (CAGR) above 5% for revenues from 2024 to 2026** [Source: BNP 2Q25 Press Release (Page 5)].

3. **Net Income Growth**:
   - The bank forecasts a **net income CAGR above 7% for 2024–2026** [Source: BNP 2Q25 Press Release (Page 5)].
   - Additionally, the **EPS growth CAGR is expected to exceed 8%** [Source: BNP 2Q25 Earnings Call Transcript (Page 3)].

4. **Operational Efficiency**:
   - The **jaws effect** (r

#### Strategic Validation


In [ ]:
question = "Management highlights their commitment to 'cost control' and achieving a 'positive jaws effect'. What specific cost-saving figures and jaws-effect metrics are provided in the reports to support this claim?"
# expected answer : Page 2. "on track to deliver cost savings of €600 million this year, with €190 million of additional savings implemented in Q2. Overall, we generated positive jaws effect of 1.7 points this quarter".
# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: Management highlights their commitment to 'cost control' and achieving a 'positive jaws effect'. What specific cost-saving figures and jaws-effect metrics are provided in the reports to support this claim?

Answer:
Based on the provided context, the following specific cost-saving figures and jaws-effect metrics support BNP's commitment to cost control and achieving a positive jaws effect:

1. **Cost Savings**:
   - BNP implemented **€190 million** of new cost savings in 2Q25, which is consistent with their full-year target and helps offset inflation [Source: BNP 2Q25 Earnings Call Transcript (Page 5)].
   - The company remains on track to deliver **€600 million** in cost savings for the year, with the €190 million achieved in Q2 contributing to this goal [Source: BNP 2Q25 Earnings Call Transcript (Page 2)].

2. **Jaws Effect Metrics**:
   - At the **Group level**, the jaws effect was **1.7 points** in 2Q25, exceeding the stated ambition of 

#### Driver & Anomaly Analysis


In [ ]:
# Add your advanced analysis questions here
question = "The headline net profit was down 4% year-over-year. What was the primary reason given for this decline, and how does the company frame its impact on the full-year 2024 trajectory?"
# expected answer : Page 3. "our net profit was down 4% due to a one-time low tax charge in the second quarter of 2024 without compromising our trajectory... as this tax impact ironed out over the full year 2024."
# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: The headline net profit was down 4% year-over-year. What was the primary reason given for this decline, and how does the company frame its impact on the full-year 2024 trajectory?

Answer:
The primary reason given for the 4% year-over-year decline in headline net profit was a one-time low tax charge in the second quarter of 2024. The company frames this impact as temporary, stating that it did not compromise the full-year 2024 profitability trajectory, as the tax impact was ironed out over the full year [Source: BNP 2Q25 Earnings Call Transcript (Page 3)].

The company also confirms its confidence in its financial outlook by maintaining a distribution policy of 60% and announcing an interim dividend of €2.59 per share, representing 50% of its EPS for the first half of 2025 [Source: BNP 2Q25 Earnings Call Transcript (Page 3)]. Additionally, the company expects a sharp acceleration in performance in the second half of 2025, with net income pr

#### Stakeholder Sentiment & Risk Inquiry


In [ ]:
# Add your risk assessment questions here
# Add your advanced analysis questions here
question = "During the Q&A, an analyst from CIC raised a concern about the profitability of BNP's European commercial banks. How did management respond, and what specific performance metric did they highlight for BNL in Italy?"
# expected answer : Page 9 (Q&A section). "Pierre Chédeville (CIC): ...difference in terms of profitability between your European path and some other of your competitors." and Lars Machenil's response: "if we look at BNL, for example... our internal metric, the RONE, which is close to 14% in the first half of the year".
# generated answer :
result = rag_with_citations(question, top_k=5)

print(f"Question: {result['question']}")
print("\nAnswer:")
print(result["answer"])
print(result["source_summary"])

Successfully processed 1 embeddings
Question: During the Q&A, an analyst from CIC raised a concern about the profitability of BNP's European commercial banks. How did management respond, and what specific performance metric did they highlight for BNL in Italy?

Answer:
During the Q&A, management responded to the analyst's concern about the profitability of BNP's European commercial banks by emphasizing their focus on improving profitability despite a challenging competitive environment. Specifically for BNL in Italy, they highlighted the **Return on Net Equity (RONE)**, which was close to **14% in the first half of 2025**, nearly twice the level seen in 2022. They also noted that costs for BNL were down by **1.5%** (excluding the DGS impact) and that the cost of risk had decreased from around **50 basis points to 38 basis points** in 2Q25 [Source: BNP 2Q25 Earnings Call Transcript (Page 9)].

Additionally, management mentioned that BNL's pre-tax profit had increased by **more than 30%*